In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
input_shape = (128, 128, 3)

conv_args = dict(kernel_size=(3,3), padding="same", activation="relu")

in_unet = Input(input_shape)
x = Conv2D(32, **conv_args)(in_unet)
x1 = Conv2D(32, **conv_args)(x)
x = MaxPool2D()(x1)

x = Conv2D(64, **conv_args)(x)
x2 = Conv2D(64, **conv_args)(x)
x = MaxPool2D()(x2)

x = Conv2D(128, **conv_args)(x)
x3 = Conv2D(128, **conv_args)(x)
x = MaxPool2D()(x3)

x = Conv2D(256, **conv_args)(x)
x = Conv2D(256, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x3])
x = Conv2D(128, **conv_args)(x)
x = Conv2D(128, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x2])
x = Conv2D(64, **conv_args)(x)
x = Conv2D(64, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x1])
x = Conv2D(32, **conv_args)(x)
x = Conv2D(32, **conv_args)(x)

out_unet = Conv2D(1, kernel_size=(3,3), padding="same", activation="sigmoid")(x)

unet = Model(in_unet, out_unet)

In [3]:
unet.compile(loss="binary_crossentropy", metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

In [4]:
import data_generation

train_gen = data_generation.SegmentationDataGenerator("./test_dataset/train")
test_gen = data_generation.SegmentationDataGenerator("./test_dataset/test")

Found 14896 files.
Found 4965 files.


In [5]:
unet.fit(train_gen, validation_data=, epochs=5)

Epoch 1/5
465/465 [==============================] - 121s 243ms/step - loss: 0.3361
Epoch 2/5
465/465 [==============================] - 68s 147ms/step - loss: 0.1467
Epoch 3/5
465/465 [==============================] - 70s 151ms/step - loss: 0.1100
Epoch 4/5
465/465 [==============================] - 71s 152ms/step - loss: 0.0842
Epoch 5/5
465/465 [==============================] - 71s 153ms/step - loss: 0.0755


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (Temp/ipykernel_23924/3571122038.py, line 1)

In [17]:
X, y = test_gen.__getitem__(1)

In [18]:
y_pred = unet.predict(X)

In [19]:
y_pred.shape

(32, 128, 128, 1)

In [ ]:
w, h = 4, 4
fig, ax = plt.subplots(h, w, figsize=(12,12))

for i in range(h):
    for j in range(w):
        ax[i,j].imshow(X[w*i+j])
        # ax[i,j].imshow(y[w*i+j,:,:,0], cmap="gray")
        ax[i,j].imshow(np.ma.masked_less(y_pred[w*i+j,:,:,0], .5), vmin=0, vmax=1, alpha=.5)
        ax[i,j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
unet = keras.models.load_model("trained_models/Unet/unet_v1.tf")

In [ ]:
image, mask = train_gen.next()
mask_pred = unet.predict(image)

In [ ]:
train_gen = ImageStream("test_dataset/test")
X_test, y_test = train_gen.load_dataset(2000)

In [ ]:
unet.evaluate()

In [ ]:
(mask_pred[w*i+j,:,:,0] > .5).sum()

In [ ]:
w, h = 4, 4
fig, ax = plt.subplots(h, w, figsize=(12,12))

for i in range(h):
    for j in range(w):
        ax[i,j].imshow(image[w*i+j])
        # ax[i,j].imshow(mask[w*i+j,:,:,0], cmap="gray")
        ax[i,j].imshow(np.ma.masked_less(mask_pred[w*i+j,:,:,0], .5), alpha=.5)
        ax[i,j].axis('off')

plt.tight_layout()
plt.show()

In [22]:
unet.save(r"trained_models\Unet\unet_v2.tf")

INFO:tensorflow:Assets written to: trained_models\Unet\unet_v2.tf\assets
